In [1]:
!pip install --upgrade openai --quiet

In [2]:
with open('key', 'r') as file:
    openai_key = file.read()

In [3]:
import openai
client = openai.OpenAI(api_key=openai_key)

In [4]:
from openai.types import Model, ModelDeleted

In [5]:
models = client.models.list()

In [6]:
type(models)

openai.pagination.SyncPage[Model]

In [7]:
for k in models:
    if "gpt" in k.id:
        print(k.id)

gpt-4-vision-preview
gpt-3.5-turbo-0613
gpt-3.5-turbo-instruct-0914
gpt-4-1106-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-3.5-turbo-16k
gpt-3.5-turbo-instruct
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-1106
gpt-4
gpt-4-0613
gpt-3.5-turbo-16k-0613


In [8]:
model="gpt-4-turbo-preview"

In [9]:
def get_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [10]:
text = f"""
step 1: extract keypoints and descriptors from all images
step 2: find matches between all images
step 3: select a reference image
step 4: begin adding the other images together with the reference image by
  first finding a transformation between them
  and then blending the images together"""

In [11]:
prompt = f"""
I'm coding in python and using the OpenCV library. Give me a python script to stitch 4 images together to create a panorama image. Use the following steps if you can:
```{text}```
"""

response = get_completion(prompt)
print(response)

Creating a panorama from multiple images involves several steps, including feature detection, feature matching, estimating a homography matrix, and finally stitching and blending the images together. Below is a Python script that demonstrates these steps using OpenCV to stitch 4 images into a panorama. This script assumes you have OpenCV installed (`pip install opencv-python`) and your images are named `image1.jpg`, `image2.jpg`, `image3.jpg`, and `image4.jpg`.

```python
import cv2
import numpy as np

# Function to stitch images
def stitch_images(images):
    # Step 1: Extract keypoints and descriptors from all images
    sift = cv2.SIFT_create()
    keypoints_descriptors = [sift.detectAndCompute(image, None) for image in images]
    
    # Step 2: Find matches between all images
    # Using FLANN based matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, se